In [1]:
!pip install PyQt5


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import numpy as np
import sys
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QPainter, QPen, QBrush
from PyQt5.QtCore import Qt, QPoint
from IPython import get_ipython
import matplotlib as mp

In [15]:
def cubic_hermite(P0, P1, M0, M1, t):
    """
    Given a pair of points and a pair of vectors, compute the cubic Hermite
    polynomial they define and evaluate it at time t.
    """
    h00 = 2*t**3 - 3*t**2 + 1
    h10 = t**3 - 2*t**2 + t
    h01 = -2*t**3 + 3*t**2
    h11 = t**3 - t**2
    return h00 * P0 + h10 * M0 + h01 * P1 + h11 * M1
def estimate_derivatives_catmull_rom(P):
    """
    Assuming P are the points of a closed curve, use the Catmull-Rom
    technique to estimate the derivative at each point of P by looking at its
    neighbors.
    """
    derivatives = np.zeros_like(P, dtype=float)
    derivatives[1:-1] = (P[:-2] - P[2:]) / 2
    derivatives[-1] = (P[-2]-P[0])/2
    derivatives[0] = (P[-1] - P[1])/2
    derivatives*=-1
    return derivatives
def catmull_rom_interpolation(P, t):
    """
    P is a set of points, which we assume are corresponded to equally spaced
    times between 0 and 1. t is a scalar value between 0 and 1. Return the
    value of the catmull-rom interpolation of P at time t. You may use
    cubic_hermite.
    """
    derivatives=estimate_derivatives_catmull_rom(P)
    n = P.shape[0]-1  # Integer division to get the number of segments    nt=n*t;
    nt=n*t;
    nt,seg=np.modf(nt)
    seg=seg.astype(int)
    # p0=P[seg-1]
    # p1=P[seg]
    # m0=derivatives[seg-1]
    # m1=derivatives[seg]
    if seg == 0:
        p0 = P[0]  # First point, can't go back further
        m0 = derivatives[0]  # First derivative
    else:
        p0 = P[seg-1]
        m0 = derivatives[seg-1]

    if seg >= len(P) - 1:
        p1 = P[-1]  # Last point, can't go forward further
        m1 = derivatives[-1]  # Last derivative
    else:
        p1 = P[seg]
        m1 = derivatives[seg]
    return cubic_hermite(p0,p1,m0,m1,nt);
    pass


In [16]:

"""randompoints = np.array([[25,25],
                        [25,100],
                        [45, 125]])
newlist = []

for i in th:
    #print(catmull_rom_interpolation(randompoints, (1-i)))
    newvertex = catmull_rom_interpolation(randompoints, (1-i))
    newlist.append(QPoint(newvertex[0].astype(int), newvertex[1].astype(int)))"""
    

def curvepositions(Qplist):
    points = []
    #points = np.empty((0,2), dtype=int)
    newlist = []
    th = np.linspace(0, 1, 1000)
    for i in Qplist:
        points.append([i.x(), i.y()])
    array = np.array((points))
    for i in th:
        newvertex = catmull_rom_interpolation(array, (1-i))
        newlist.append(QPoint(newvertex[0].astype(int), newvertex[1].astype(int)))
    return newlist


In [17]:
def CircularSegments(P1,P2,P3,P4):

    x1=P1.x()
    y1=P1.y()
    x2=P3.x()
    y2=P3.y()
    
    x12=P2.x()
    y12=P2.y()
    x22=P4.x()
    y22=P4.y()

    m1= (y12-y1)/(x12-x1)
    m2= (y22-y2)/(x22-x2)


    h = (y1-y2 + (x2/m2) - (x1/m1))/(1/m1)-(1/m2)
    k=y1- (1/m1)*(h-x1)
    r=np.sqrt((x1-h)**2 + (y1-k)**2)
    angles = np.linspace(0, 2*np.pi, 1000)
    points=[]
    for angle in angles:
        x=h+r*np.cos(angle)
        y=k+r*np.sin(angle)
        points.append(QPoint(x.astype(int),y.astype(int)))

    return points

#points=[QPoint(50, 20), QPoint(60, 30),QPoint(70, 100),QPoint(55, 35)]
#CircularSegments(points[0],points[1],points[2],points[3])

In [22]:
def ArcGenerator(P1,P2,P3):

#Point 1 is our point on tangent line    
    xT=P1.x()
    yT=P1.y()
    x1=P2.x()
    y1=P2.y()
    x2=P3.x()
    y2=P3.y()


    if yT-y1 == 0:
        print('dividing by zero error')
        m=0
    elif xT-x1 == 0:
        print('dividing by zero error')
        m=0
    else:
        m= -1/((yT-y1)/(xT-x1))

    # Midpoint P2 and P3
    mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2

    # Equation of the perpendicular line through P1 (since tangent slope is m, perpendicular slope is -1/m)
    a1, b1 = -m, 1
    c1 = a1 * x1 + b1 * y1

    # Equation of the perpendicular bisector of P1P2
    a2, b2 = x2 - x1, y2 - y1
    c2 = a2 * mid_x + b2 * mid_y

    # Solve the system of linear equations to find the center (h, k)
    A = np.array([[a1, b1], [a2, b2]])
    B = np.array([c1, c2])
    h, k = np.linalg.solve(A, B)

    # Calculate the radius
    r = np.sqrt((x1 - h)**2 + (y1 - k)**2)

    points=[]

    if (np.arctan2(P2.y() - k, P2.x() - h) < np.arctan2(P3.y() - k, P3.x() - h)):
        angles = np.linspace(np.arctan2(P2.y() - k, P2.x() - h), np.arctan2(P3.y() - k, P3.x() - h), 1000)
        
    else:
        angles = np.linspace(np.arctan2(P3.y() - k, P3.x() - h), np.arctan2(P2.y() - k, P2.x() - h), 1000)
        
    for angle in angles:
        x=h+r*np.cos(angle)
        y=k+r*np.sin(angle)
        points.append(QPoint(x.astype(int),y.astype(int)))

    return points


In [23]:
#dubins path

def dubins_path(P1,P2,P3,P4):
    xT=P1.x()
    yT=P1.y()
    x1=P2.x()
    y1=P2.y()
    x2=P3.x()
    y2=P3.y()
    xT2=P4.x()
    yT2=P4.y()
    
    #tangent case
    #check for tangency

    points=[P2,P3]
    #return tangent segments
    th=np.linspace(0,1,1000)
    for t in th:
        x=(1-t)*x1 + t*x2
        y=(1-t)*y1 + t*y2
        points.append(QPoint(x.astype(int),y.astype(int)))
    

    return points








In [24]:
class VertexWidget(QWidget):
    def __init__(self):
        super().__init__()
        self.vertices = [QPoint(20, 20), QPoint(40, 100)]
        self.selected_vertex_index = None
        self.setWindowTitle("Vertex Mover")
        self.setGeometry(100, 100, 400, 300)
        self.show()
    def paintEvent(self, event):
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setPen(QPen(Qt.black, 2))
        painter.setBrush(QBrush(Qt.blue, Qt.SolidPattern))
        for vertex in self.vertices:
            painter.drawEllipse(vertex, 10, 10)
        #for vertexnumber in range (len(self.vertices)-1):
            #painter.drawLine(self.vertices[vertexnumber], self.vertices[vertexnumber+1])


        """
        every 3 create an arc
        """
        arcList=[]
        dubinsList=[]
        i=0
        if( len(self.vertices) >= 3):
            while i < (len(self.vertices)-3):
                arcList= (ArcGenerator(self.vertices[i],self.vertices[i+1],self.vertices[i+2]))
                i+=3
                for vertexnumber in range (len(arcList)-1):
                    painter.drawLine(arcList[vertexnumber], arcList[vertexnumber+1])

       
        if(len(self.vertices)>=5):
            i=0
            while i < (len(self.vertices)-5):
                dubinsList = dubins_path(self.vertices[i+1],self.vertices[i+2],self.vertices[i+4],self.vertices[i+5])
                i+=3
                for vertexnumber in range (len(dubinsList)-1):
                    painter.drawLine(dubinsList[vertexnumber], dubinsList[vertexnumber+1])

        
    
    
    def mousePressEvent(self, event):
        for index, vertex in enumerate(self.vertices):
            if (event.pos() - vertex).manhattanLength() < 10:
                self.selected_vertex_index = index
                break
        if event.button() == Qt.MouseButton.RightButton:
            self.vertices.append(event.pos())
            self.update()
    def keyPressEvent(self, event):
        key = event.key()
        if key == Qt.Key_Space:
            self.vertices = self.vertices[:-1]
            self.update()
    def mouseMoveEvent(self, event):
        if self.selected_vertex_index is not None:
            self.vertices[self.selected_vertex_index] = event.pos()
            self.update()
    def mouseReleaseEvent(self, event):
        self.selected_vertex_index = None
def run_app():
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    widget = VertexWidget()
    app.exec_()




# Use the '%matplotlib qt' magic to enable the Qt event loop integration
get_ipython().run_line_magic('matplotlib', 'qt')
# Run the app
run_app()

dividing by zero error
dividing by zero error
dividing by zero error
